# Gender Classification Using SVM

## Importing The Main Libraries

In [86]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

## Loading The Dataset

In [87]:
data = pd.read_csv('gender_classification_v7.csv')

In [88]:
data.head()

,long_hair,forehead_width_cm,forehead_height_cm,nose_wide,nose_long,lips_thin,distance_nose_to_lip_long,gender
0,1,11.8,6.1,1,0,1,1,Male
1,0,14.0,5.4,0,0,1,0,Female
2,0,11.8,6.3,1,1,1,1,Male
3,0,14.4,6.1,0,1,1,1,Male
4,1,13.5,5.9,0,0,0,0,Female


## EDA

## main information

In [89]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5001 entries, 0 to 5000
Data columns (total 8 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   long_hair                  5001 non-null   int64  
 1   forehead_width_cm          5001 non-null   float64
 2   forehead_height_cm         5001 non-null   float64
 3   nose_wide                  5001 non-null   int64  
 4   nose_long                  5001 non-null   int64  
 5   lips_thin                  5001 non-null   int64  
 6   distance_nose_to_lip_long  5001 non-null   int64  
 7   gender                     5001 non-null   object 
dtypes: float64(2), int64(5), object(1)
memory usage: 312.7+ KB


In [90]:
data.isna().sum()

,0
long_hair,0
forehead_width_cm,0
forehead_height_cm,0
nose_wide,0
nose_long,0
lips_thin,0
distance_nose_to_lip_long,0
gender,0


In [91]:
data.describe()

,long_hair,forehead_width_cm,forehead_height_cm,nose_wide,nose_long,lips_thin,distance_nose_to_lip_long
count,5001.000000,5001.000000,5001.000000,5001.000000,5001.000000,5001.000000,5001.000000
mean,0.869626,13.181484,5.946311,0.493901,0.507898,0.493101,0.498900
std,0.336748,1.107128,0.541268,0.500013,0.499988,0.500002,0.500049
min,0.000000,11.400000,5.100000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,12.200000,5.500000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,13.100000,5.900000,0.000000,1.000000,0.000000,0.000000
75%,1.000000,14.000000,6.400000,1.000000,1.000000,1.000000,1.000000
max,1.000000,15.500000,7.100000,1.000000,1.000000,1.000000,1.000000


In [92]:
for column in data.columns:

  print ( data[column].value_counts() , '\n' )

long_hair
1    4349
0     652
Name: count, dtype: int64 

forehead_width_cm
12.0    168
11.8    161
14.1    155
11.5    154
14.0    154
11.6    152
11.9    150
13.3    149
14.2    149
12.1    145
12.2    145
12.6    145
13.2    143
14.3    143
12.9    141
12.5    141
13.7    140
12.3    138
13.6    138
12.8    138
13.0    137
13.4    137
13.5    136
13.8    134
11.7    133
13.1    131
13.9    131
12.7    130
12.4    127
11.4     84
14.9     79
14.5     72
15.2     71
15.5     66
15.1     65
15.3     64
15.4     64
14.4     63
14.6     61
15.0     60
14.7     59
14.8     48
Name: count, dtype: int64 

forehead_height_cm
5.7    306
5.9    303
5.3    301
5.5    299
6.1    296
5.1    293
6.2    290
5.8    290
6.4    287
6.3    285
6.5    285
5.6    279
6.0    275
5.4    265
5.2    259
6.6    126
6.8    118
7.0    117
7.1    113
6.9    109
6.7    105
Name: count, dtype: int64 

nose_wide
0    2531
1    2470
Name: count, dtype: int64 

nose_long
1    2540
0    2461
Name: count, dtype: int64 

## Plots

In [93]:
for column in data.columns:
    if column != 'gender':
        fig = px.histogram(data, x=column, color='gender', title=f'Distribution of {column} by Gender')
        fig.show()

## Data Preprocessing

## Handle The Missing Data

      ❎ No missing Data

## Encoding the Categorical Columns

In [94]:
mapGender = { 'Male' : 1 , 'Female' : 0 }

data['gender'] = data['gender'].map(mapGender)

In [95]:
data['gender'].dtype

dtype('int64')

## Splitting the Data into input & output

In [96]:
x = data.drop( 'gender' , axis = 1 ).copy()
y = data['gender'].copy()

In [97]:
x.head()

,long_hair,forehead_width_cm,forehead_height_cm,nose_wide,nose_long,lips_thin,distance_nose_to_lip_long
0,1,11.8,6.1,1,0,1,1
1,0,14.0,5.4,0,0,1,0
2,0,11.8,6.3,1,1,1,1
3,0,14.4,6.1,0,1,1,1
4,1,13.5,5.9,0,0,0,0


In [98]:
y.head()

,gender
0,1
1,0
2,1
3,1
4,0


In [99]:
y.value_counts()

,count
gender,
0,2501
1,2500


## Splitting the Data into Training & Testing

In [100]:
from sklearn.model_selection import train_test_split

In [101]:
xTrain , xTest , yTrain , yTest = train_test_split ( x , y , test_size = 0.2  , random_state = 42 )

In [102]:
xTrain.head()

,long_hair,forehead_width_cm,forehead_height_cm,nose_wide,nose_long,lips_thin,distance_nose_to_lip_long
4677,1,15.4,6.6,0,1,1,1
800,1,12.2,5.2,0,0,0,0
3671,0,11.5,5.8,0,0,0,0
4193,1,12.7,5.1,0,0,0,0
2968,1,15.1,5.6,1,1,1,1


In [103]:
yTrain.head()

,gender
4677,1
800,0
3671,0
4193,0
2968,1


In [104]:
xTest.head()

,long_hair,forehead_width_cm,forehead_height_cm,nose_wide,nose_long,lips_thin,distance_nose_to_lip_long
1501,1,13.2,5.7,1,1,1,1
2586,1,13.7,6.0,0,0,0,0
2653,0,12.9,5.3,0,0,0,1
1055,1,13.2,5.9,1,1,1,1
705,0,15.2,5.6,1,1,1,1


In [105]:
yTest.head()

,gender
1501,1
2586,0
2653,0
1055,1
705,1


## Feature Scaling

      ❎ No need For Feature Scaling Here as the whole Features is binary ( 0 - 1 ) , it will not affected so much by Feature
      
        Scaling

        Only the 2 columns overhead width and height is non binary but no need to apply Feature Scaling for them as they both

        are in the same range almost

## Model Training

In [106]:
from sklearn.svm import SVC

In [108]:
classifier = SVC( kernel = 'rbf' )         # kernel = 'rbf'  ->   Default ( separate with non linear separator )

classifier.fit ( xTrain.values , yTrain.values )

SVC()

## Model Testing

In [111]:
predictedGender = classifier.predict ( xTest.values )

In [112]:
predictedGender

array([1, 0, 0, ..., 0, 1, 0])

## Model Evaluation

In [118]:
from sklearn.metrics import accuracy_score , confusion_matrix , classification_report

### Training Evaluation

In [117]:
PredictedGenderTraining = classifier.predict ( xTrain.values )

TrainingResults = pd.DataFrame ( columns = x.columns , data = xTrain )
TrainingResults['Predicted Gender'] = PredictedGenderTraining
TrainingResults['Actual Gender'] = yTrain

TrainingResults.head()

,long_hair,forehead_width_cm,forehead_height_cm,nose_wide,nose_long,lips_thin,distance_nose_to_lip_long,Predicted Gender,Actual Gender
4677,1,15.4,6.6,0,1,1,1,1,1
800,1,12.2,5.2,0,0,0,0,0,0
3671,0,11.5,5.8,0,0,0,0,0,0
4193,1,12.7,5.1,0,0,0,0,0,0
2968,1,15.1,5.6,1,1,1,1,1,1


In [119]:
cmTrain = confusion_matrix ( yTrain , PredictedGenderTraining )

cmTrain

array([[1943,   56],
       [  64, 1937]])

In [127]:
# Labels for the confusion matrix
labels = ['Actual 0', 'Actual 1']

# Create the heatmap trace
heatmap = go.Heatmap(
    z=cmTrain,
    x=['Predicted 0', 'Predicted 1'],
    y=labels,
    colorscale='Blues',
    colorbar=dict(title='Count')
)

# Add annotations for the cell values
annotations = []
for i, row in enumerate(cmTrain):
    for j, value in enumerate(row):
        annotations.append(go.layout.Annotation(
            text=str(value),
            x=['Predicted 0', 'Predicted 1'][j],
            y=labels[i],
            xref='x1',
            yref='y1',
            showarrow=False,
            font=dict(color='black', size=15)
        ))

# Create the figure and add the heatmap and annotations
fig = go.Figure(data=[heatmap])
fig.update_layout(
    title='Confusion Matrix - Training Data',
    xaxis=dict(title='Predicted Label'),
    yaxis=dict(title='True Label'),
    annotations=annotations ,
    height = 600 ,
    width = 800
)

fig.show()

In [129]:
TrainingAccuracy = accuracy_score ( yTrain , PredictedGenderTraining )

print ( f" The Training Accuracy = { TrainingAccuracy * 100 } %" )

 The Training Accuracy = 97.0 %


In [133]:

# Calculate correct and incorrect predictions
TrainingResults['Prediction_Status'] = (TrainingResults['Predicted Gender'] == TrainingResults['Actual Gender']).map({True: 'Correct', False: 'Incorrect'})

# Get the value counts and reset index to create a DataFrame
prediction_counts_df = TrainingResults['Prediction_Status'].value_counts().reset_index()

# Rename the columns to something Plotly Express expects or is less ambiguous
prediction_counts_df.columns = ['Status_Category', 'Count']

# Plotting the distribution of prediction status
fig = px.bar(prediction_counts_df,
             x='Status_Category',
             y='Count',
             title='Training Prediction Status (Correct vs. Incorrect)',
             labels={'Status_Category': 'Prediction Status', 'Count': 'Number of Samples'},
             color='Status_Category',
             color_discrete_map={'Correct': 'green', 'Incorrect': 'red'})
fig.show()

## Testing Evaluation

In [134]:
TestingResults = pd.DataFrame ( columns = x.columns , data = xTest )
TestingResults['Predicted Gender'] = predictedGender
TestingResults['Actual Gender'] = yTest

TestingResults.head()

,long_hair,forehead_width_cm,forehead_height_cm,nose_wide,nose_long,lips_thin,distance_nose_to_lip_long,Predicted Gender,Actual Gender
1501,1,13.2,5.7,1,1,1,1,1,1
2586,1,13.7,6.0,0,0,0,0,0,0
2653,0,12.9,5.3,0,0,0,1,0,0
1055,1,13.2,5.9,1,1,1,1,1,1
705,0,15.2,5.6,1,1,1,1,1,1


In [136]:
cmTest = confusion_matrix ( yTest , predictedGender )

cmTest

array([[486,  16],
       [ 21, 478]])

In [137]:
# Labels for the confusion matrix
labels = ['Actual 0', 'Actual 1']

# Create the heatmap trace
heatmap = go.Heatmap(
    z=cmTest,
    x=['Predicted 0', 'Predicted 1'],
    y=labels,
    colorscale='Blues',
    colorbar=dict(title='Count')
)

# Add annotations for the cell values
annotations = []
for i, row in enumerate(cmTest):
    for j, value in enumerate(row):
        annotations.append(go.layout.Annotation(
            text=str(value),
            x=['Predicted 0', 'Predicted 1'][j],
            y=labels[i],
            xref='x1',
            yref='y1',
            showarrow=False,
            font=dict(color='black', size=15)
        ))

# Create the figure and add the heatmap and annotations
fig = go.Figure(data=[heatmap])
fig.update_layout(
    title='Confusion Matrix - Testing Data',
    xaxis=dict(title='Predicted Label'),
    yaxis=dict(title='True Label'),
    annotations=annotations,
    height = 600,
    width = 800
)

fig.show()

In [143]:
TestAccuracy = accuracy_score ( yTest , predictedGender )

print ( f" The Testing Accuracy = { (TestAccuracy * 100):.2f} % " )

 The Testing Accuracy = 96.30 % 


In [144]:
# Calculate correct and incorrect predictions for testing data
TestingResults['Prediction_Status'] = (TestingResults['Predicted Gender'] == TestingResults['Actual Gender']).map({True: 'Correct', False: 'Incorrect'})

# Get the value counts and reset index to create a DataFrame
prediction_counts_df_test = TestingResults['Prediction_Status'].value_counts().reset_index()

# Rename the columns to something Plotly Express expects or is less ambiguous
prediction_counts_df_test.columns = ['Status_Category', 'Count']

# Plotting the distribution of prediction status for testing data
fig = px.bar(prediction_counts_df_test,
             x='Status_Category',
             y='Count',
             title='Testing Prediction Status (Correct vs. Incorrect)',
             labels={'Status_Category': 'Prediction Status', 'Count': 'Number of Samples'},
             color='Status_Category',
             color_discrete_map={'Correct': 'green', 'Incorrect': 'red'})
fig.show()